# Label BoundingBox —— Image & Video
Pre-require: [ipywidgets](https://github.com/jupyter-widgets/ipywidgets), you can install it by executing `pip install ipywidgets` in terminal.   
Usage: Replace `RECORD` & `IMGs_ROOT` with path & directory that you want to explore, run the cell, then choose images from the `Dropdown` box.

环境要求：
- 需要[ipywidgets](https://github.com/jupyter-widgets/ipywidgets)，可以使用终端命令`pip install ipywidgets`安装。

用法：
1. 先用fix_records_length.ipynb文件将不定长记录重新整理成定长记录
2. 将变量`RECORD`的值修改为定长记录文件的路径
3. 将变量`IMGs_ROOT`的值修改为记录文件中路径的相对起始位置（如果是绝对路径，则将`IMGs_ROOT`设为空字符串`''`）
4. 如果需要自动保存，将变量`AUTOSAVE`设置为`True`（这将会占用更多资源，如果数据过多，建议设为`False`）
4. 运行代码块，在`Label`文本框中输入新的label后，回车即自动加载下一张图片
5. 最后一定要**点击保存按钮**，确保结果覆盖保存到`RECORD`文件

> **随时保存！随时保存！随时保存！**重要的事说三遍！

In [1]:
# Change RECORD as the fixed records file's path
RECORD = "./jilu.txt"
# Change IMGs_ROOT as the dictionary where images are 
IMGs_ROOT = "./"
# Save changes after every input
AUTOSAVE = True
# Standing time for each image when AutoPlay, unit: second
LOOP_TIME = 0.2

IMAGE_FMT = [
    '.jpg', '.jpeg', '.jpe', '.png', '.bmp', '.dib', '.tif', '.tiff', '.webp',
    '.pbm', '.pgm', '.ppm', '.sr', '.ras', '.exr', '.jp2'
]
VIDEO_FMT = ['.mp4', '.avi', '.mpg', '.mpeg', '.mov']

import os
import cv2
import numpy as np
from time import sleep
from ipywidgets import Label, Text, Button, Dropdown, Image, HBox, HTML
from IPython.display import display

idx = 0
metadata = []
info_lbl = HTML()
rec_txt = Text(description="RECORD:")
rec_txt.layout.width = "90%"
jmp_btn = Button(description="Jump", button_style='primary')
jmp_btn.layout.width = "80px"
drpbox = Dropdown(description="[0/0]:")
drpbox.layout.width = "90%"
prev_btn = Button(description="Preview")
next_btn = Button(description="Next")
loop_btn = Button(description="AutoPlay")
loop_btn.layout.width = "200px"
save_btn = Button(description="Save")
imgbox = Image(format='jpg')
label_txt = Text(description="Label:")
label_lbl = Label()


def read_rec(_=None):
    global RECORD, idx, metadata
    idx = 0
    RECORD = rec_txt.value
    if os.path.isfile(RECORD):
        with open(RECORD) as fp:
            for itm in fp.readlines():
                inpt = itm.split()
                if len(inpt) % 6 != 1:
                    continue
                try:
                    path = inpt[0]
                    # Prework with path field
                    path = path.replace('/nfs/syzhou/github/yolo3_hand/', '')
                    bboxes = np.transpose(
                        np.array([float(d) for d in inpt[1:]]).reshape(6, -1))
                except Exception as e:
                    info_lbl.value = "<font color=\"#ff000000\">{}</font>".format(
                        "Error: %s!" % str(e))
                for bbx in bboxes:
                    bbx = bbx.tolist()
                    bbx.insert(0, path)
                    metadata.append(bbx)
    else:
        metadata = []
        info_lbl.value = "<font color=\"#ff000000\">{}</font>".format(
            "Error: %s is not available directory!" % IMGs_ROOT)
    drpbox.options = [(value, i) for i, value in enumerate(metadata)]
    show_pic()


def show_pic():
    global idx, metadata
    prev_btn.disabled = next_btn.disabled = False
    if idx <= 0:
        idx = 0
        prev_btn.disabled = True
    if idx >= len(metadata) - 1:
        idx = len(metadata) - 1
        next_btn.disabled = True
    drpbox.description = "[{}/{}]:".format(idx + 1, len(metadata))
    if len(metadata) == 0:
        return None
    drpbox.value = idx
    pic_path, x0, y0, x1, y1, label, score = metadata[drpbox.index]
    crtpath = os.path.join(IMGs_ROOT, pic_path)
    _, ext = os.path.splitext(crtpath)
    if ext in IMAGE_FMT:
        img = cv2.imread(crtpath)
        img = cv2.rectangle(img, (int(x0), int(y0)), (int(x1), int(y1)),
                            [0, 0, 255], 2)
        label_lbl.value = 'Preview Label: ' + str(label)
        label_txt.value = ''
        imgbox.value = cv2.imencode('.jpg', img)[1].tobytes()
    elif ext in VIDEO_FMT:
        cap = cv2.VideoCapture(crtpath)
        fps = cap.get(cv2.CAP_PROP_FPS)
        while (cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            imgbox.value = cv2.imencode('.jpg', frame)[1].tobytes()
            sleep(1 / fps)
    return crtpath


def choose_pic(attval):
    global idx
    idx = attval['new']
    show_pic()


def prev_pic(sender):
    global idx
    idx -= 1
    show_pic()


def next_pic(sender):
    global idx
    idx += 1
    show_pic()


def label_enter(sender):
    global idx
    # todo: check input label
    metadata[idx][-2] = sender.value
    if AUTOSAVE:
        save_pic(sender)
    next_pic(sender)


def loop_pic(sender):
    if loop_btn.description == "AutoPlay":
        loop_btn.description = "Stop loop by Kernel->Interrupt"
        loop_btn.disabled = True
        global idx
        try:
            while idx <= len(metadata) - 1:
                show_pic()
                sleep(0.2)
                idx += 1
        except KeyboardInterrupt:
            pass
    loop_btn.description = "AutoPlay"
    loop_btn.disabled = False


def save_pic(sender):
    try:
        with open(RECORD, 'w+') as fo:
            for itm in metadata:
                fo.write(str(itm) + '\n')
        info_lbl.value = "<font color=\"#00ff0000\">{}</font>".format(
            "Saved into %s" % RECORD)
    except Exception as e:
        info_lbl.value = "<font color=\"#ff000000\">{}</font>".format(
            "Error: %s!" % str(e))


rec_txt.on_submit(read_rec)
jmp_btn.on_click(read_rec)
drpbox.observe(choose_pic, names='value')
prev_btn.on_click(prev_pic)
next_btn.on_click(next_pic)
label_txt.on_submit(label_enter)
loop_btn.on_click(loop_pic)
save_btn.on_click(save_pic)
display(HBox([rec_txt, jmp_btn]), drpbox,
        HBox([prev_btn, next_btn, loop_btn, save_btn]), info_lbl, imgbox,
        HBox([label_txt, label_lbl]))

# init
rec_txt.value = RECORD
read_rec()

HBox(children=(Text(value='', description='RECORD:', layout=Layout(width='90%')), Button(button_style='primary', description='Jump', layout=Layout(width='80px'), style=ButtonStyle())))

Dropdown(description='[0/0]:', layout=Layout(width='90%'), options=(), value=None)

HBox(children=(Button(description='Preview', style=ButtonStyle()), Button(description='Next', style=ButtonStyle()), Button(description='AutoPlay', layout=Layout(width='200px'), style=ButtonStyle()), Button(description='Save', style=ButtonStyle())))

HTML(value='')

Image(value=b'', format='jpg')

HBox(children=(Text(value='', description='Label:'), Label(value='')))